In [1]:
import pandas as pd
import numpy as np

The methodology follows 4 main steps: 
- 1) Import production data per facility previously collected
- 2) Import, clean and harmonize environmental data, which typically include Supply-Use-Tables (SUT), National Pollutant Inventory, GHG from large facilities and satellite data.
- 3) Normalize the emissions per facility by the production 
- 4) Create consumption market by weighting the normalized emissions per facility based on the market share by facility

# 1 - Import production data per facility

# 2 - Import, clean and harmonize environmental data

## 2.1 Import and clean

### 2.1.1 NPRI

#### Create mapping NPRI-EI through IW+ mapping

In [2]:
# Import existing concordances 
mapping_npri_iw = pd.read_excel(r'data/Concordances/openIO_IW_EI_concordance.xlsx', sheet_name='NPRI_to_IW21')
mapping_ei_iw = pd.read_excel(r'data/Concordances/openIO_IW_EI_concordance.xlsx', sheet_name='EI_to_IW+')

In [3]:
# Standardize column names for merging
mapping_npri_iw.rename(columns={'IMPACT World+ flows': 'iw_name'}, inplace=True)
mapping_ei_iw.rename(columns={'iw name': 'iw_name'}, inplace=True)

In [4]:
# Add the 'EI' column to df_npri_iw based on matching iw_name with df_ei_iw
mapping_npri_iw['EI'] = mapping_npri_iw['iw_name'].map(
    lambda iw: mapping_ei_iw[mapping_ei_iw['iw_name'] == iw]['ecoinvent name'].iloc[0]
    if iw in mapping_ei_iw['iw_name'].values else 'No match found'
)


In [5]:
mapping_npri_iw

,OpenIO flows,iw_name,EI
0,Carbon dioxide,"Carbon dioxide, fossil","Carbon dioxide, fossil"
1,Methane,"Methane, fossil",Methane
2,Dinitrogen monoxide,Dinitrogen monoxide,Dinitrogen monoxide
3,CF4,"Methane, tetrafluoro-, CFC-14","Methane, tetrafluoro-, R-14"
4,C2F6,"Ethane, hexafluoro-, HFC-116","Ethane, hexafluoro-, HFC-116"
...,...,...,...
348,Propylene oxide,Propylene oxide,Propylene oxide
349,Thiourea,Thiourea,No match found
350,2-Ethoxyethyl acetate,2-Ethoxyethyl acetate,No match found
351,Azo disperse dyes,NaN,No match found


In [6]:
#mapping_npri_iw.to_excel(r'data/Concordances/NPRI_EI_concordance.xlsx', index=False)

#### Import and sort NPRI data

In [7]:
npri_df = pd.read_excel(r'data/Emissions_accounts/Raw_data/NPRI/NPRI-INRP_DataDonnées_2023.xlsx', sheet_name='INRP-NPRI 2023', skiprows=3)

In [8]:
npri_df

,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,Province,Postal Code,...,Organic Compound Recovery,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling"
0,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000020
1,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004000
2,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097000
3,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000090
4,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63869,2023,34009,ZF Group Oakville,ZF Group Oakville,Oakville,Oakville,Toronto,Toronto,ON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.735000
63870,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000
63871,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000
63872,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000


In [9]:
# We need to rename it to merge it to NPRI data
mapping_npri_iw.rename(columns={'OpenIO flows': 'Substance Name (English)'}, inplace=True)

In [10]:
# We add the Ecoinvent name based on the mapping 
npri_df = npri_df.merge(mapping_npri_iw[['Substance Name (English)', 'EI']],
              on='Substance Name (English)',
              how='left')

In [11]:
npri_df

,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,Province,Postal Code,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
0,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000020,Acrolein
1,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004000,Ammonia
2,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097000,"Carbon monoxide, fossil"
3,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000090,Formaldehyde
4,2023,30809,•\tEmboutissage Takumi Canada Inc./Takumi Stam...,Takumi Stamping St. Thomas,St. Thomas,St. Thomas,London,London,ON,N5P 0B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Manganese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63869,2023,34009,ZF Group Oakville,ZF Group Oakville,Oakville,Oakville,Toronto,Toronto,ON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.735000,NaN
63870,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000,No match found
63871,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000,"Particulates, < 2.5 um"
63872,2023,7095,Zochem ULC,Zochem,Brampton,Brampton,Toronto,Toronto,ON,L6T 3T4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.356000,No match found


In [12]:
metal_mining_df = npri_df[npri_df['NAICS 4 Sector Name (English)'] == 'Metal ore mining']
metal_manufacturing_df = npri_df[npri_df["NAICS 4 Code"].astype(str).str[:3] == "331"]

In [13]:
# To see how much NPRI substances do not found a match in Ecoinvent 
print((metal_mining_df['EI'].eq('No match found').mean() * 100)) 
print((metal_manufacturing_df['EI'].eq('No match found').mean() * 100)) 

16.16945107398568
21.282327586206897


So respectively 86% and 79% of the substances for the metal ore mining and primary metal manufacturing are matched

In [15]:
# To have the main information and get a template for site-specific data
recap_metal_mining_df = metal_mining_df[['NPRI ID', 'Company Name', 'Facility Name', 'City', 
 'CSD', 'Province', 
 'Latitude', 'Longitude', 
 'NAICS 4 Sector Name (English)', 'NAICS 4 Code', 
'NAICS 6 Sector Name (English)', 'NAICS 6 Code']].drop_duplicates()

In [16]:
# To have the main information and get a template for site-specific data
recap_metal_manufacturing_df = metal_manufacturing_df[['NPRI ID', 'Company Name', 'Facility Name', 'City', 
 'CSD', 'Province', 
 'Latitude', 'Longitude', 
 'NAICS 4 Sector Name (English)', 'NAICS 4 Code', 
'NAICS 6 Sector Name (English)', 'NAICS 6 Code']].drop_duplicates()

#### Analysis per company and facility

In [18]:
# Define the category mapping included in the columns of the Excel file 
category_mapping = {
    'Air Emissions': ['Stack Emissions', 'Storage / Handling', 'Fugitive Emissions', 'Spills', 'Other', 'Total'],
    'Water Releases': ['Direct Discharge', 'Spills', 'Leaks', 'Total', 'Receiving Waterbody (English)', 'Receiving Waterbody (French)'],
    'Land Releases': ['Spills', 'Leaks', 'Other', 'Total'],
    'Total Releases': ['Total Releases (Excluding Road Dust)', 'Road Dust Emissions', 'Total Releases Including Road Dust'],
    'On-Site Disposal': ['Land Fill', 'Land Treatment', 'Underground Injection', 'Tailings', 'Waste Rock', 'Total On-Site'],
    'Off-Site Disposal': ['Land Fill', 'Land Treatment', 'Underground Injection', 'Storage', 'Tailings', 'Waste Rock', 'Total Off-Site'],
    'Transfers for Treatment': ['Physical Treatment', 'Chemical Treatment', 'Biological Treatment', 'Incineration', 'Transfer to a Municipal Sewage Treatment Plant', 'Total'],
    'Total On/Off Treatment': ['Total'],
    'Transfers for Recycling': [
        'Energy Recovery', 'Solvent Recovery', 'Organic Compound Recovery', 'Metal Recovery',
        'Inorganic Compound Recovery', 'Acid or Base Recovery', 'Catalyst Recovery',
        'Pollution Abatement Residue Recovery', 'Used Oil Recovery', 'Other', 'Total'
    ],
    'Grand Total': ['Total Releases, Disposals and Transfers for Recycling']
}

In [27]:
def summarize_emissions_by_facility(df, category_mapping):
    """
    Summarize emissions for all big categories as separate columns with company, facility, and city details.
    """
    results = df[['NPRI ID', 'Company Name', 'Facility Name', 'City', 'CSD', 'NAICS 4 Sector Name (English)', 'NAICS 4 Code',  'NAICS 6 Sector Name (English)', "NAICS 6 Code"]].copy()
    for category, columns in category_mapping.items():
        valid_columns = [col for col in columns if col in df.columns]
        if valid_columns:
            # Sum emissions for the current category
            df[category] = df[valid_columns].apply(pd.to_numeric, errors='coerce').fillna(0).sum(axis=1)
            results[category] = df[category]
    # Group by facility details and sum the emissions for all categories
    results = results.groupby(['NPRI ID', 'Company Name', 'Facility Name', 'City', 'CSD', 'NAICS 4 Sector Name (English)', 'NAICS 4 Code',  'NAICS 6 Sector Name (English)', "NAICS 6 Code"], as_index=False).sum()
    return results

In [28]:
facility_df = summarize_emissions_by_facility(npri_df, category_mapping)

In [29]:
facility_metal_mining_df = facility_df[facility_df['NAICS 4 Sector Name (English)'] == 'Metal ore mining']
facility_metal_manufacturing_df = facility_df[facility_df["NAICS 4 Code"].astype(str).str[:3] == "331"]

In [30]:
facility_metal_mining_df

,NPRI ID,Company Name,Facility Name,City,CSD,NAICS 4 Sector Name (English),NAICS 4 Code,NAICS 6 Sector Name (English),NAICS 6 Code,Air Emissions,Water Releases,Land Releases,Total Releases,On-Site Disposal,Off-Site Disposal,Transfers for Treatment,Total On/Off Treatment,Transfers for Recycling,Grand Total
9,99,Les Mines Agnico Eagle Limitée,Division Laronde,Rouyn-Noranda,Preissac,Metal ore mining,2122,Gold and silver ore mining,212220,344.571500,863.663150,273.895750,4546.080300,892955.780000,446477.890000,172.285750,172.285750,273.895750,448750.930150
71,394,BARRICK GOLD INC.,Nickel Plate Mine,Penticton,Okanagan-Similkameen G,Metal ore mining,2122,Gold and silver ore mining,212220,0.000000,0.000000,0.000000,67.138000,0.000000,0.000000,0.000000,0.000000,0.000000,33.569000
196,1147,Cameco Corporation,Rabbit Lake Operation,Saskatoon,"Division No. 18, Unorganized",Metal ore mining,2122,Uranium ore mining,212291,494.926000,250.348000,247.463000,946.462000,2646.740000,1323.370000,247.463000,247.463000,2137.409000,3686.547000
197,1148,Cameco Corporation,Key Lake Operation,Saskatoon,"Division No. 18, Unorganized",Metal ore mining,2122,Uranium ore mining,212291,733.476120,562.818920,421.858060,1072.511840,404494.502000,202247.251000,420.816060,420.816060,2091.134060,204560.938920
198,1149,Cameco Corporation,McArthur River Operation,Saskatoon,"Division No. 18, Unorganized",Metal ore mining,2122,Uranium ore mining,212291,110.566600,69.519300,56.585300,238.776600,0.000000,0.000000,56.585300,56.585300,2054.585300,2119.992300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5086,33049,Prodigy Gold Inc.,Magino Mine,Dubreuilville,"Algoma, Unorganized, North Part",Metal ore mining,2122,Gold and silver ore mining,212220,1023.246602,516.523316,511.623301,2694.248798,476617.840536,238308.920268,511.623301,511.623301,511.623301,239656.044667
5261,33819,Red Lake Madsen Mine Ltd.,Red Lake Madsen Mine,Madsen,Red Lake,Metal ore mining,2122,Gold and silver ore mining,212220,9.165642,46.559535,4.582821,95.354362,20229.423200,10114.711600,4.582821,4.582821,4.582821,10162.388781
5289,33878,B2Gold Back River Corp.,Goose Property,Goose Lake,"Kitikmeot, Unorganized",Metal ore mining,2122,Gold and silver ore mining,212220,1454.769000,727.384500,727.384500,1737.840600,2424.057400,1212.029900,727.384500,727.384500,727.384500,2080.949600
5397,34668,Iamgold Corporation,Fayolle,Rouyn-Noranda,Rouyn-Noranda,Metal ore mining,2122,Gold and silver ore mining,212220,0.000000,0.000000,0.000000,97.542000,0.000000,0.000000,0.000000,0.000000,0.000000,48.771000


#### Template for data collection

In [34]:
# We add air, water and land emissions to the template
emissions_columns = ['Facility Name', 'Air Emissions', 'Water Releases', 'Land Releases']

recap_metal_mining_df = pd.merge(
    recap_metal_mining_df, facility_metal_mining_df[emissions_columns], on='Facility Name', how='left'
)
recap_metal_manufacturing_df = pd.merge(
    recap_metal_manufacturing_df, facility_metal_manufacturing_df[emissions_columns], on='Facility Name', how='left'
)

In [35]:
recap_metal_mining_df

,NPRI ID,Company Name,Facility Name,City,CSD,Province,Latitude,Longitude,NAICS 4 Sector Name (English),NAICS 4 Code,NAICS 6 Sector Name (English),NAICS 6 Code,Air Emissions,Water Releases,Land Releases
0,11623,1911 Gold Corporation,True North Gold Mine,Bissett,"Division No. 19, Unorganized",MB,51.021800,-95.679500,Metal ore mining,2122,Gold and silver ore mining,212220,2.176000,1.088000,1.088000
1,1568,Agnico Eagle Mines,Macassa Mine,Kirkland Lake,Kirkland Lake,ON,48.130320,-80.087340,Metal ore mining,2122,Gold and silver ore mining,212220,674.604440,529.169340,337.302220
2,25155,Agnico Eagle Mines Limited,Hope Bay Project,Cambridge Bay,"Kitikmeot, Unorganized",NU,68.039900,-106.574464,Metal ore mining,2122,Gold and silver ore mining,212220,1239.798076,619.899038,796.539038
3,25188,Agnico Eagle Mines Limited (MAIN),Mine Canadian Malartic,Malartic,Malartic,QC,48.124507,-78.129272,Metal ore mining,2122,Gold and silver ore mining,212220,3628.942200,1940.931800,1984.111100
4,24216,Agnico Eagle Mines Ltd.,Division Meadowbank,Baker Lake,"Keewatin, Unorganized",NU,65.014900,-96.052400,Metal ore mining,2122,Gold and silver ore mining,212220,8117.675826,4075.681021,4063.458641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",NL,56.337780,-62.094400,Metal ore mining,2122,Nickel-copper ore mining,212232,8025.111442,4040.531181,4014.318721
100,32238,Victoria Gold Corp.,Eagle Gold Mine,Vancouver,"Yukon, Unorganized",YT,64.025450,-135.834300,Metal ore mining,2122,Gold and silver ore mining,212220,1498.472400,749.236200,1157.616000
101,10010,Wesdome Gold Mines Ltd.,Eagle River Mine & Mill Complex,Wawa,"Thunder Bay, Unorganized",ON,47.985620,-85.461160,Metal ore mining,2122,Gold and silver ore mining,212220,6.397006,3.905922,3.198503
102,11123,Wesdome Gold Mines Ltd.,Eagle River Mine Site,Wawa,"Thunder Bay, Unorganized",ON,47.986300,-85.461400,Metal ore mining,2122,Gold and silver ore mining,212220,503.497284,251.748642,251.748642


### 2.1.2 - GHG from large facilities

In [38]:
ghg_facility_df = pd.read_csv(
    r'data/Emissions_accounts/Raw_data/GHG_large_facilities/Greenhouse gas emissions from large facilities.csv',
    encoding='ISO-8859-1')

In [39]:
ghg_facility_df

,Facility ID,Facility name,Company name,City,Address,Postal code,Province,Latitude,Longitude,Total emissions,Units,Report year,Industry classification,Industry classification link,Facility information,Facility details
0,10001,Division Alma,Produits forestiers Résolu,Alma,1100 Melançon Street,G8B 5W2,Quebec,48.56500,-71.65556,39.08,kilotonnes of carbon dioxide equivalents (kt C...,2022,Mechanical Pulp Mills,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
1,10002,Aciérie - ArcelorMittal Contrecoeur,ArcelorMittal Produits Longs Canada s.e.n.c,Contrecoeur,0 3900 Route des Aciéries,J0L 1C0,Quebec,45.82055,-73.26362,NaN,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron and Steel Mills and Ferro-Alloy Manufactu...,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
2,10003,"Foothills Pipeline, Alberta",Foothills Pipe Lines Ltd.,Airdrie,NaN,T4A 2G7,Alberta,0.00000,0.00000,352.66,kilotonnes of carbon dioxide equivalents (kt C...,2022,Pipeline Transportation of Natural Gas,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
3,10004,Kingston CoGen,Validus Power Corp,Bath,5146 Taylor-Kidd Boulevard,K0H 1G0,Ontario,44.20950,-76.72460,1.01,kilotonnes of carbon dioxide equivalents (kt C...,2022,Fossil-Fuel Electric Power Generation,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
4,10005,AGC Flat glass North America ltée,AGC Flat glass North America ltée,St Augustin De Desmaures,250 rue de Copenhague,G3A 2H3,Quebec,0.00000,0.00000,NaN,kilotonnes of carbon dioxide equivalents (kt C...,2022,Glass Manufacturing,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,12717,Lieu d'enfouissement technique de la Ville de ...,Ville de Matane,Matane,330 Yves-Bérubé Street,G4W 3M6,Quebec,48.81885,-67.57371,21.39,kilotonnes of carbon dioxide equivalents (kt C...,2022,Waste Treatment and Disposal,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
2625,12718,Lieu d'enfouissement technique de la ville de ...,ville de Sept-Iles,Sept Iles,750 du lac Daigle Road Northeast,NaN,Quebec,50.26970,-66.29690,50.36,kilotonnes of carbon dioxide equivalents (kt C...,2022,Waste Treatment and Disposal,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
2626,12719,Pure Sunfarms Delta 3,Pure Sunfarms,Delta,4431 80th Street,V4K 3N3,British Columbia,49.08432,-122.99768,10.08,kilotonnes of carbon dioxide equivalents (kt C...,2022,Cannabis grown under cover,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
2627,12720,KARMAX HEAVY STAMPING - MILTON,Magna International Inc.,Milton,333 Market Drive,L9T 4Z7,Ontario,43.52070,-79.91130,10.85,kilotonnes of carbon dioxide equivalents (kt C...,2022,Motor Vehicle Metal Stamping,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...


In [40]:
# We need to sort it by NAICS list, since they are not properly listed by "hierarchy", e.g. no parent nor code

In [41]:
# Define classification lists
metal_ore_mining_naics = [
    "Metal ore mining",
    "Iron ore mining",
    "Gold and silver ore mining",
    "Copper, nickel, lead and zinc ore mining",
    "Lead-zinc ore mining",
    "Nickel-copper ore mining",
    "Copper-zinc ore mining",
    "Other metal ore mining",
    "Uranium ore mining",
    "All other metal ore mining",
    "Non-metallic mineral mining and quarrying",
    "Stone mining and quarrying",
    "Granite mining and quarrying",
    "Limestone mining and quarrying",
    "Marble mining and quarrying",
    "Sandstone mining and quarrying",
    "Sand, gravel, clay, and ceramic and refractory minerals mining and quarrying",
    "Sand and gravel mining and quarrying",
    "Shale, clay and refractory mineral mining and quarrying",
    "Other non-metallic mineral mining and quarrying",
    "Diamond mining",
    "Salt mining",
    "Asbestos mining",
    "Gypsum mining",
    "Potash mining",
    "Peat extraction",
    "All other non-metallic mineral mining and quarrying",
    "Support activities for mining, and oil and gas extraction",
    "Oil and gas contract drilling",
    "Contract drilling (except oil and gas)",
    "Services to oil and gas extraction",
    "Other support activities for mining"
]

metal_manufacturing_naics = [
    "Primary metal manufacturing",
    "Iron and steel mills and ferro-alloy manufacturing",
    "Steel product manufacturing from purchased steel",
    "Iron and steel pipes and tubes manufacturing from purchased steel",
    "Rolling and drawing of purchased steel",
    "Cold-rolled steel shape manufacturing",
    "Steel wire drawing",
    "Alumina and aluminum production and processing",
    "Primary production of alumina and aluminum",
    "Aluminum rolling, drawing, extruding and alloying",
    "Non-ferrous metal (except aluminum) production and processing",
    "Non-ferrous metal (except aluminum) smelting and refining",
    "Copper rolling, drawing, extruding and alloying",
    "Non-ferrous metal (except copper and aluminum) rolling, drawing, extruding and alloying",
    "Foundries",
    "Ferrous metal foundries",
    "Iron foundries",
    "Steel foundries",
    "Non-ferrous metal foundries",
    "Non-ferrous metal die-casting foundries",
    "Non-ferrous metal foundries (except die-casting)"
]

In [42]:
def filter_ghg_facility_naics(df, classifications):
    # Normalize the classifications to lowercase for case-insensitive comparison
    classifications_lower = [cls.lower() for cls in classifications]
    
    df_copy = df.copy()
    df_copy['NAICS_Lower'] = df_copy['Industry classification'].str.lower()
    filtered_df = df_copy[df_copy['NAICS_Lower'].isin(classifications_lower)]
    filtered_df = filtered_df.drop(columns=['NAICS_Lower'])
    filtered_df = filtered_df.reset_index(drop=True)
    
    return filtered_df

In [43]:
ghg_facility_df_metal_mining = filter_ghg_facility_naics(ghg_facility_df, metal_ore_mining_naics)
ghg_facility_df_metal_manufacturing = filter_ghg_facility_naics(ghg_facility_df, metal_manufacturing_naics)

In [44]:
ghg_facility_df_metal_manufacturing

,Facility ID,Facility name,Company name,City,Address,Postal code,Province,Latitude,Longitude,Total emissions,Units,Report year,Industry classification,Industry classification link,Facility information,Facility details
0,10002,Aciérie - ArcelorMittal Contrecoeur,ArcelorMittal Produits Longs Canada s.e.n.c,Contrecoeur,0 3900 Route des Aciéries,J0L 1C0,Quebec,45.82055,-73.26362,NaN,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron and Steel Mills and Ferro-Alloy Manufactu...,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
1,10010,Rio Tinto Alcan Inc.,Rio Tinto Alcan Inc,Kitimat,1 Smeltersite Road,V8C 2H2,British Columbia,54.00286,-128.69531,420.39,kilotonnes of carbon dioxide equivalents (kt C...,2022,Primary Production of Alumina and Aluminum,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
2,10011,Algoma Steel Inc,Algoma Steel Inc.,Sault Ste. Marie,105 West Street North,P6A 7B4,Ontario,46.52170,-84.36370,3936.24,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron and Steel Mills and Ferro-Alloy Manufactu...,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
3,10012,Aluminerie Alouette inc.,Aluminerie Alouette inc.,SeptÎles,400 Chemin de de la Pointe-Noire,G4R 5M9,Quebec,50.15830,-66.44160,1140.90,kilotonnes of carbon dioxide equivalents (kt C...,2022,Primary Production of Alumina and Aluminum,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
4,10013,Aluminerie de Baie-Comeau,Ne pas prendre - Alcoa Canada Co.,Baie Comeau,0 100 Maritime Route,G4Z 2L6,Quebec,49.25830,-68.14610,534.69,kilotonnes of carbon dioxide equivalents (kt C...,2022,Primary Production of Alumina and Aluminum,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,12301,GRIFFIN CANADA - WINNIPEG,Griffin Canada Inc.,Winnipeg,2500 Day Street,R2C 3A4,Manitoba,49.91300,-97.00250,NaN,kilotonnes of carbon dioxide equivalents (kt C...,2022,Steel Foundries,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
69,12554,Molycop Canada Plant 2,Moly-Cop Canada,Kamloops,300 Andover Crescent,V2C 6X2,British Columbia,50.65598,-120.09703,NaN,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron Foundries,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
70,12555,Moly-Cop Canada,Moly-Cop Canada,Kamloops,250 Andover Crescent,V2C 6X2,British Columbia,50.65726,-120.09649,25.12,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron Foundries,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...
71,12579,Fonderie Laperle,Canada Pipe Co ltd,Saint Ours,106 montée de la Basse,J0G 1P0,Quebec,45.89470,-73.13870,0.51,kilotonnes of carbon dioxide equivalents (kt C...,2022,Iron Foundries,https://www23.statcan.gc.ca/imdb/p3VD.pl?Funct...,https://climate-change.canada.ca/facility-emis...,http://indicators-map.canada.ca/App/Detail?id=...


### 2.1.3 NRCan's 900A

In [45]:
nrcan_mining_df = pd.read_excel(r'data/NRCan/Principal Mineral Areas, Producing Mines, and Oil and Gas Fields (900A)/producing_mines.xlsx')
nrcan_manufacturing_df = pd.read_excel(r'data/NRCan/Principal Mineral Areas, Producing Mines, and Oil and Gas Fields (900A)/metal_work.xlsx')

In [46]:
nrcan_mining_df

,OPERATIO_E,OPERATIO_F,OWNER_E,OWNER_F,FAC_TYPE_E,FAC_TYPE_F,FAC_DESC_E,FAC_DESC_F,COMMODIT_E,COMMODIT_F,COM_DESC_E,COM_DESC_F,COM_GROU_E,COM_GROU_F,CITY_E,CITY_F,PROVINCE_E,PROVINCE_F,LATITUDE,LONGITUDE
0,Sleeping Giant,Géant Dormant,Abcourt Mines Inc.,Mines Abcourt Inc.,(C.),(Con.),Concentrator,Concentrateur,"Au, Ag","Au, Ag","Gold, silver","Or, argent",Precious metals,Métaux précieux,north of Amos,au nord d’Amos,Quebec,Québec,49.132590,-77.974340
1,Amaruq,Amaruq,Agnico Eagle Mines Limited,Mines Agnico Eagle Limitée,"(P., U.)","(C.O., Sout.)","Open-pit, underground","Ciel ouvert, souterraine",Au,Au,Gold,Or,Precious metals,Métaux précieux,Baker Lake,Baker Lake,Nunavut,Nunavut,65.415000,-96.697000
2,Canadian Malartic,Canadian Malartic,Agnico Eagle Mines Limited,Mines Agnico Eagle Limitée,"(P., C.)","(C.O., Con.)","Open-pit, concentrator","Ciel ouvert, concentrateur","Au, Ag","Au, Ag","Gold, silver","Or, argent",Precious metals,Métaux précieux,Malartic,Malartic,Quebec,Québec,48.122223,-78.130824
3,Detour Lake,Detour Lake,Agnico Eagle Mines Limited,Mines Agnico Eagle Limitée,"(P., C.)","(C.O., Con.)","Open-pit, concentrator","Ciel ouvert, concentrateur",Au,Au,Gold,Or,Precious metals,Métaux précieux,Northeast of Cochrane,au nord-est de Cochrane,Ontario,Ontario,50.018143,-79.717631
4,Goldex,Goldex,Agnico Eagle Mines Limited,Mines Agnico Eagle Limitée,"(U., C.)","(Sout., Con.)","Underground, concentrator","Souterraine, concentrateur","Au, Ag","Au, Ag","Gold, silver","Or, argent",Precious metals,Métaux précieux,Val-d’Or,Val-d’Or,Quebec,Québec,48.092600,-77.873300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Mishi,Mishi,Wesdome Gold Mines Ltd.,Mines d’Or Wesdome Ltée,(P.),(C.O.),Open-pit,Ciel ouvert,Au,Au,Gold,Or,Precious metals,Métaux précieux,Wawa,Wawa,Ontario,Ontario,48.110280,-85.452500
195,Coal Valley,Coal Valley,Westmoreland Coal Company,Westmoreland Coal Company,(P.),(C.O.),Open-pit,Ciel ouvert,Coal (thermal),Charbon (thermique),Coal (thermal),Charbon (thermique),Coal,Charbon,Edson,Edson,Alberta,Alberta,53.072796,-116.786616
196,Estevan,Estevan,Westmoreland Coal Company,Westmoreland Coal Company,(P.),(C.O.),Open-pit,Ciel ouvert,Coal (thermal),Charbon (thermique),Coal (thermal),Charbon (thermique),Coal,Charbon,Bienfait,Bienfait,Saskatchewan,Saskatchewan,49.087329,-102.864956
197,Genesee,Genesee,Westmoreland Coal Company,Westmoreland Coal Company,(P.),(C.O.),Open-pit,Ciel ouvert,Coal (thermal),Charbon (thermique),Coal (thermal),Charbon (thermique),Coal,Charbon,Genesee,Genesee,Alberta,Alberta,53.323876,-114.267236


In [47]:
nrcan_manufacturing_df

,OPERATIO_E,OPERATIO_F,OWNER_E,OWNER_F,FAC_TYPE_E,FAC_TYPE_F,COM_GROU_E,COM_GROU_F,COM_DESC_E,COM_DESC_F,PROVINCE_E,PROVINCE_F,LATITUDE,LONGITUDE
0,Baie-Comeau,Baie-Comeau,Alcoa Corporation,Alcoa Corporation,Smelter,Usine de fusion,Aluminum,Aluminum,Aluminum (pure or alloyed),Aluminium (pur ou allié),Quebec,Québec,49.256200,-68.149100
1,Deschambault,Deschambault,Alcoa Corporation,Alcoa Corporation,Smelter,Usine de fusion,Aluminum,Aluminum,Aluminum (pure),Aluminium (pur),Quebec,Québec,46.693900,-71.946700
2,Bécancour,Bécancour,Alcoa Corporation and Rio Tinto Aluminum Inc.,Alcoa Corporation et Rio Tinto Aluminum Inc.,Smelter,Usine de fusion,Aluminum,Aluminum,Aluminum (pure or alloyed),Aluminium (pur ou allié),Quebec,Québec,46.381000,-72.385200
3,Algoma Steel,Algoma Steel,Algoma Steel Inc.,Algoma Steel Inc.,Basic oxygen furnace,Convertisseur basique à oxygène,Iron ore,Minerai de fer,Steel,Acier,Ontario,Ontario,46.520550,-84.366640
4,AltaSteel,AltaSteel,AltaSteel Inc.,AltaSteel Inc.,Electric arc furnace,Four électrique à arc,Iron ore,Minerai de fer,Steel,Acier,Alberta,Alberta,53.492783,-113.387962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Sault Ste. Marie Division,Division Sault Ste. Marie,Triple M Metal LP,Triple M Metal LP,NaN,NaN,NaN,NaN,NaN,NaN,Ontario,Ontario,46.524326,-84.395602
69,Welland Mill,Aciérie de Welland,Valbruna ASW Inc.,Valbruna ASW Inc.,Electric arc furnace,Four électrique à arc,Iron ore,Minerai de fer,Steel,Acier,Ontario,Ontario,42.998265,-79.235065
70,Copper Cliff Complex,Complexe Copper Cliff,Vale Canada Limited,Vale Canada Limitée,"Smelter, refinery, plant","Usine de fusion, affinerie, usine",Base metals,Métaux communs,"Nickel (oxide sinter, pellets, powder, sulfide...","Nickel (sinters d’oxydes, boulettes, poudre, s...",Ontario,Ontario,46.480100,-81.056900
71,Port Colborne,Port Colborne,Vale Canada Limited,Vale Canada Limitée,Refinery,Affinerie,Base metals,Métaux communs,"Electrolytic cobalt, platinum group metals (in...","Cobalt (production électrolytique), métaux du ...",Ontario,Ontario,42.883000,-79.240000


### 2.1.4 MinCan database from Dallaire-Fortin (2024)

In [66]:
min_can = pd.read_excel(r'data/Mining_production_data/MinCan _Past and Present Productive Mines of Canada, 1950-2022_March2024.xlsx', sheet_name='Data')
min_can

,company1,company2,company3,company4,company5,company6,namemine,town,province,latitude,...,commodity6,commodity7,commodity8,information,source1,source2,source3,link1,link2,link3
0,Quebec Copper Corp. Ltd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quebec,45.263786,...,NaN,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN
1,Noland Mines Ltd.,NaN,NaN,NaN,NaN,NaN,Spruce Creek,Atlin,British Columbia,59.382720,...,NaN,NaN,NaN,NaN,0,1,1,https://www.mindat.org/feature-6088912.html,https://www.mindat.org/loc-257276.html,https://mrdata.usgs.gov/ardf/show-ardf.php?ard...
2,Falconbridge Nickel Mines Ltd.,NaN,NaN,NaN,NaN,NaN,Wesfrob (Tasu),Moresby Island,British Columbia,52.758340,...,NaN,NaN,NaN,NaN,0,1,1,https://www.mindat.org/loc-26169.html,NaN,NaN
3,Greenwood Coal Co. Ltd.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nova Scotia,45.550219,...,NaN,NaN,NaN,The currently entered coordinates are for the ...,0,1,1,https://www.mindat.org/loc-290980.html,NaN,NaN
4,Noranda Mines Ltd,Normetal Mining Corporation Ltd.,Falconbridge Ltd.,Xstrata,NaN,NaN,Horne,Noranda,Quebec,48.254270,...,Pyrite,NaN,NaN,Alternative names: Horne Mine (1975) The mine ...,0,1,1,https://www.mindat.org/loc-609.html,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,"Imperial Metals Corporation, Mitsubishi Materi...",Teck Resources Limited,Huckleberry Mines Ltd.,NaN,NaN,NaN,Huckleberry,Kemano,British Columbia,53.681110,...,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN
943,BHP Diamonds Inc.,Dia Met Minerals Ltd.,Charles Fipke and Stewart Blusson,Arctic Canadian Diamond Company,NaN,NaN,Panda (Ekati Project),Lac de Gras,Northwest Territories,64.716667,...,NaN,NaN,NaN,Brief closure between 2020 and 2021,1,0,0,NaN,NaN,NaN
944,Magnola Magnesium Inc.,Society generale de financement du Quebec,NaN,NaN,NaN,NaN,Ardorbec,Danville,Quebec,45.748462,...,NaN,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN
945,Canadian Johns-Manville Co. Ltd.,Asbestos Inc.,NaN,NaN,NaN,NaN,Jeffrey,Asbestos (Val-des-Sources),Quebec,45.775100,...,NaN,NaN,NaN,"FR: En 1992, la mine est l'objet d'une importa...",0,0,1,https://miningwatch.ca/sites/default/files/asb...,https://niche-canada.org/2015/11/19/workers-as...,https://www.researchgate.net/publication/28664...


In [73]:
def is_mine_active(row):
    current_year = 2022  # As mentioned in the explanation
    
    # Convert year values to integers, ignoring non-numeric values
    def to_int(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    
    # Convert all year columns to integers
    open1 = to_int(row['open1'])
    close1 = to_int(row['close1'])
    open2 = to_int(row['open2'])
    close2 = to_int(row['close2'])
    open3 = to_int(row['open3'])
    close3 = to_int(row['close3'])
    
    # Check if any of the 'close' columns have the value 'open'
    if row['close1'] == 'open' or row['close2'] == 'open' or row['close3'] == 'open':
        return 'Active'
    
    # Find the latest year among open and close columns
    years = [open1, close1, open2, close2, open3, close3]
    years = [year for year in years if year is not None]
    
    if not years:
        return 'Unknown'
    
    latest_year = max(years)
    
    # If the latest year is a 'close' year, the mine is inactive
    if latest_year in [close1, close2, close3]:
        return 'Inactive'
    
    # If the latest year is an 'open' year and it's the current year or later, consider it active
    if latest_year in [open1, open2, open3] and latest_year >= current_year:
        return 'Active'
    
    # For all other cases, consider it inactive
    return 'Inactive'

In [74]:
# Apply the function to create a new column 'mine_status'
min_can['mine_status'] = min_can.apply(is_mine_active, axis=1)

In [75]:
min_can['mine_status'].value_counts()

mine_status
Inactive    776
Active      171
Name: count, dtype: int64

In [78]:
min_can = min_can[min_can['mine_status'] == 'Active']

In [79]:
min_can

,company1,company2,company3,company4,company5,company6,namemine,town,province,latitude,...,commodity7,commodity8,information,source1,source2,source3,link1,link2,link3,mine_status
7,Dominion Magnesium Ltd.,Chromasco Corporation Ltd.,Timminco Limited,NaN,NaN,NaN,Haley,Haley,Ontario,45.598880,...,NaN,NaN,NaN,0,1,1,http://www.geologyontario.mndm.gov.on.ca/mndmf...,https://www.mindat.org/loc-14742.html,NaN,Active
13,Noranda Inc.,Brunswick Mining and Smelting Corporation Limi...,NaN,NaN,NaN,NaN,Stratmat (Halfmile),NaN,New Brunswick,47.305278,...,NaN,NaN,indigenous participation,0,1,1,https://www.nrcan.gc.ca/sites/www.nrcan.gc.ca/...,NaN,NaN,Active
18,Steep Rock Iron Mines Ltd.,OMYA (Canada) Inc.,NaN,NaN,NaN,NaN,Calcite Division,Perth,Ontario,45.149689,...,NaN,NaN,NaN,0,1,1,http://www.geologyontario.mndm.gov.on.ca/mndmf...,NaN,NaN,Active
23,Marietta Resources International Ltd.,Lacana Mines Inc.,Zemex Corporation,Suzorite Mica products of Boucherville,NaN,NaN,Parent Mica Deposit,Suzor,Quebec,47.938412,...,NaN,NaN,"Changed name in Lacana Mines Inc. ""In late 198...",0,1,1,https://cmscontent.nrs.gov.bc.ca/geoscience/Pu...,https://gq.mines.gouv.qc.ca/documents/examine/...,NaN,Active
24,Baskatong Quartz Products Ltd.,Sitec Quartz Inc.,SKW Canada Inc.,Hogan Holdings Inc.,NaN,NaN,Charlevoix,Charlevoix (Saint-Urbain),Quebec,47.749900,...,NaN,NaN,NaN,0,1,1,https://gq.mines.gouv.qc.ca/documents/examine/...,https://www.globenewswire.com/news-release/201...,NaN,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,BP Ressources Canada Ltd.,Lhoist,NaN,NaN,NaN,NaN,NaN,Fort Langley,British Columbia,49.189659,...,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,Active
932,Continental Lime Ltd.,Graymont Western Canada Inc.,NaN,NaN,NaN,NaN,NaN,Exshaw,Alberta,51.071389,...,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,Active
935,Red Deer Silica Inc.,NaN,NaN,NaN,NaN,NaN,NaN,Hudson Bay,Saskatchewan,52.985830,...,NaN,NaN,small scale /// Lower Cretaceous Mannville Gro...,1,0,0,NaN,NaN,NaN,Active
943,BHP Diamonds Inc.,Dia Met Minerals Ltd.,Charles Fipke and Stewart Blusson,Arctic Canadian Diamond Company,NaN,NaN,Panda (Ekati Project),Lac de Gras,Northwest Territories,64.716667,...,NaN,NaN,Brief closure between 2020 and 2021,1,0,0,NaN,NaN,NaN,Active


## 2.2 Merge NPRI, GHG and NRCan datasets

We want to automatically add information from the GHG for large facility and NRCan datasets to the cleaned NPRI data. 

In [80]:
def merge_datasets_by_coordinates(df1, df2, df3, df4, atol=0.01):
    """
    Merge three DataFrames (df1, df2, df3) by matching geographic coordinates (latitude and longitude).
    Adds additional columns for matched facility names, company names, and other relevant columns.

    Parameters:
        df1 (DataFrame): The main DataFrame containing 'Latitude' and 'Longitude'.
        df2 (DataFrame): The secondary DataFrame containing 'Latitude', 'Longitude', 'Facility name',
                         'Company name', 'Total emissions', and 'Facility ID'.
        df3 (DataFrame): The tertiary DataFrame containing 'Latitude', 'Longitude', 'Facility name',
                         'Company name', 'FAC_DESC_E', and 'COM_DESC_E'.
        atol (float): Absolute tolerance for coordinate matching. Default is 0.01 (~1.1 km).

    Returns:
        DataFrame: A copy of df1 with additional columns for matched data from df2 and df3.
    """
    # Standardize column names for consistency
    df1 = df1.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}).copy()
    df2 = df2.rename(columns={"Latitude": "latitude", "Longitude": "longitude", "Facility name": "facility_name", "Company name": "company_name"}).copy()
    df3 = df3.rename(columns={"LATITUDE": "latitude", "LONGITUDE": "longitude", "OPERATIO_E": "facility_name", "OWNER_E": "company_name"}).copy()
    df4 = df4.copy()

    # Initialize additional columns for coordinates and metadata from df2 and df3
    df1['latitude_df2'] = "no coordinates matching found"
    df1['longitude_df2'] = "no coordinates matching found"
    df1['latitude_df3'] = "no coordinates matching found"
    df1['longitude_df3'] = "no coordinates matching found"
    df1['facility_name_df2'] = None
    df1['company_name_df2'] = None
    df1['facility_name_df3'] = None
    df1['company_name_df3'] = None
    df1['FAC_TYPE_E'] = None
    df1['COM_DESC_E'] = None
    df1['Total emissions'] = None
    df1['Facility ID'] = None
    df1['link1'] = None
    df1['link2'] = None

    # Match and add coordinates and metadata from df2
    def match_coordinates_df2(row):
        matching_row = df2[
            (np.isclose(df2['latitude'], row['latitude'], atol=atol)) & 
            (np.isclose(df2['longitude'], row['longitude'], atol=atol))
        ]
        if not matching_row.empty:
            match = matching_row.iloc[0]
            return [
                match['latitude'],
                match['longitude'],
                match['facility_name'],
                match['company_name'],
                match['Total emissions'],
                match['Facility ID']
            ]
        return ["no coordinates matching found", "no coordinates matching found", None, None, None, None]

    df1[['latitude_df2', 'longitude_df2', 'facility_name_df2', 'company_name_df2', 'Total emissions', 'Facility ID']] = df1.apply(
        match_coordinates_df2, axis=1, result_type='expand')

    # Match and add coordinates and metadata from df3
    def match_coordinates_df3(row):
        matching_row = df3[
            (np.isclose(df3['latitude'], row['latitude'], atol=atol)) & 
            (np.isclose(df3['longitude'], row['longitude'], atol=atol))
        ]
        if not matching_row.empty:
            match = matching_row.iloc[0]
            return [
                match['latitude'],
                match['longitude'],
                match['facility_name'],
                match['company_name'],
                match['FAC_TYPE_E'],
                match['COM_DESC_E']
            ]
        return ["no coordinates matching found", "no coordinates matching found", None, None, None, None]

    df1[['latitude_df3', 'longitude_df3', 'facility_name_df3', 'company_name_df3', 'FAC_TYPE_E', 'COM_DESC_E']] = df1.apply(
        match_coordinates_df3, axis=1, result_type='expand')
    
    # Match and add coordinates and metadata from df4
    def match_coordinates_df4(row):
        matching_row = df4[
            (np.isclose(df4['latitude'], row['latitude'], atol=atol)) & 
            (np.isclose(df4['longitude'], row['longitude'], atol=atol))
        ]
        if not matching_row.empty:
            match = matching_row.iloc[0]
            return [
                match['latitude'],
                match['longitude'],
                match['link1'],
                match['link2'],
                match['link3'],
            ]
        return ["no coordinates matching found", "no coordinates matching found", None, None, None]

    df1[['latitude_df4', 'longitude_df4', 'link1', 'link2', 'link3']] = df1.apply(
        match_coordinates_df4, axis=1, result_type='expand')
    
    # Calculate match percentages
    matches_df2 = df1['latitude_df2'].ne("no coordinates matching found").sum()
    matches_df3 = df1['latitude_df3'].ne("no coordinates matching found").sum()
    matches_df4 = df1['latitude_df4'].ne("no coordinates matching found").sum()
    total_rows = len(df1)

    match_percentage_df2 = (matches_df2 / total_rows) * 100
    match_percentage_df3 = (matches_df3 / total_rows) * 100
    match_percentage_df4 = (matches_df4 / total_rows) * 100

    print(f"Match Percentage between df1 and df2: {match_percentage_df2:.2f}%")
    print(f"Match Percentage between df1 and df3: {match_percentage_df3:.2f}%")
    print(f"Match Percentage between df1 and df4: {match_percentage_df4:.2f}%")
    
    df1.rename(
        columns={
    "Company Name": "Company_Name_NPRI", 
    "Facility Name": "Facility_Name_NPRI",
    "latitude": "Latitude_NPRI",
    "longitude": "Longitude_NPRI",
    "latitude_df2": "Latitude_GHG",
    "longitude_df2": "Longitude_GHG",
    "latitude_df3": "Latitude_NRCan",
    "longitude_df3": "Longitude_NRCan",
    "facility_name_df2": "Facility_Name_GHG",
    "company_name_df2": "Company_Name_GHG",
    "facility_name_df3": "Facility_Name_NRCan",
    "company_name_df3": "Company_Name_NRCan",
    "FAC_TYPE_E": "Mining_Processing_Type", 
    "COM_DESC_E": "Commodities",
    "Total emissions": "GHGE (ktCO2eq)",
    "Facility ID": "GHG ID",
    "link1": "Link1_MinCan",
    "link2": "Link2_MinCan",
    "link3": "Link3_MinCan"
        }, inplace=True
    )

    return df1

In [82]:
mining_df = merge_datasets_by_coordinates(
    recap_metal_mining_df, ghg_facility_df_metal_mining, nrcan_mining_df, min_can,
    atol=0.1)

Match Percentage between df1 and df2: 63.46%
Match Percentage between df1 and df3: 70.19%
Match Percentage between df1 and df4: 53.85%


In [84]:
mining_df

,NPRI ID,Company_Name_NPRI,Facility_Name_NPRI,City,CSD,Province,Latitude_NPRI,Longitude_NPRI,NAICS 4 Sector Name (English),NAICS 4 Code,...,Company_Name_NRCan,Mining_Processing_Type,Commodities,GHGE (ktCO2eq),GHG ID,Link1_MinCan,Link2_MinCan,latitude_df4,longitude_df4,Link3_MinCan
0,11623,1911 Gold Corporation,True North Gold Mine,Bissett,"Division No. 19, Unorganized",MB,51.021800,-95.679500,Metal ore mining,2122,...,None,None,None,NaN,NaN,None,None,no coordinates matching found,no coordinates matching found,None
1,1568,Agnico Eagle Mines,Macassa Mine,Kirkland Lake,Kirkland Lake,ON,48.130320,-80.087340,Metal ore mining,2122,...,Agnico Eagle Mines Limited,"(U., C.)","Gold, silver",24.25,10617.0,http://www.geologyontario.mndm.gov.on.ca/mndmf...,https://publications.gc.ca/collections/Collect...,48.130261,-80.088542,NaN
2,25155,Agnico Eagle Mines Limited,Hope Bay Project,Cambridge Bay,"Kitikmeot, Unorganized",NU,68.039900,-106.574464,Metal ore mining,2122,...,None,None,None,34.50,10681.0,NaN,NaN,68.137186,-106.612652,NaN
3,25188,Agnico Eagle Mines Limited (MAIN),Mine Canadian Malartic,Malartic,Malartic,QC,48.124507,-78.129272,Metal ore mining,2122,...,Agnico Eagle Mines Limited,"(P., C.)","Gold, silver",194.87,10669.0,NaN,NaN,48.129852,-78.100285,NaN
4,24216,Agnico Eagle Mines Ltd.,Division Meadowbank,Baker Lake,"Keewatin, Unorganized",NU,65.014900,-96.052400,Metal ore mining,2122,...,Agnico Eagle Mines Limited,(C.),Gold,248.77,10587.0,NaN,NaN,65.025,-96.04889,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",NL,56.337780,-62.094400,Metal ore mining,2122,...,Vale Newfoundland and Labrador Limited,"(P., U., C.)","Nickel, copper, cobalt",148.26,10570.0,NaN,NaN,56.331344,-62.097057,NaN
100,32238,Victoria Gold Corp.,Eagle Gold Mine,Vancouver,"Yukon, Unorganized",YT,64.025450,-135.834300,Metal ore mining,2122,...,Victoria Gold Corporation,"(P., C.)",Gold,51.74,12459.0,NaN,NaN,64.0322,-135.8456,NaN
101,10010,Wesdome Gold Mines Ltd.,Eagle River Mine & Mill Complex,Wawa,"Thunder Bay, Unorganized",ON,47.985620,-85.461160,Metal ore mining,2122,...,Wesdome Gold Mines Ltd.,"(U., C.)",Gold,NaN,NaN,https://miningdataonline.com/property/232/Eagl...,NaN,47.98439,-85.45937,NaN
102,11123,Wesdome Gold Mines Ltd.,Eagle River Mine Site,Wawa,"Thunder Bay, Unorganized",ON,47.986300,-85.461400,Metal ore mining,2122,...,Wesdome Gold Mines Ltd.,"(U., C.)",Gold,NaN,NaN,https://miningdataonline.com/property/232/Eagl...,NaN,47.98439,-85.45937,NaN


In [83]:
manufacturing_df = merge_datasets_by_coordinates(
    recap_metal_manufacturing_df, ghg_facility_df_metal_manufacturing, nrcan_manufacturing_df, min_can,
    atol=0.1)

Match Percentage between df1 and df2: 53.94%
Match Percentage between df1 and df3: 50.30%
Match Percentage between df1 and df4: 8.48%


In [85]:
column_order = [
    "NPRI ID",
    "GHG ID",
    "City",
    "CSD",
    "Province",
    "Latitude_NPRI",
    "Longitude_NPRI",
    "NAICS 4 Sector Name (English)",
    "NAICS 4 Code",
    "NAICS 6 Sector Name (English)",
    "NAICS 6 Code",
    "Company_Name_NPRI",
    "Facility_Name_NPRI",
    "Air Emissions",
    "Water Releases",
    "Land Releases",
    "GHGE (ktCO2eq)",
    "Mining_Processing_Type",
    "Commodities",
    "Link1_MinCan",
    "Link2_MinCan",
    "Link3_MinCan"
]


In [86]:
mining_df = mining_df[[col for col in column_order if col in mining_df.columns]]
manufacturing_df = manufacturing_df[[col for col in column_order if col in manufacturing_df.columns]]

In [87]:
with pd.ExcelWriter("data/Mining_production_data/template_data_collection_from_npri.xlsx", engine='openpyxl') as writer:
    mining_df.to_excel(writer, sheet_name='metal_mining', index=False)
    manufacturing_df.to_excel(writer, sheet_name='metal_manufacturing', index=False)

In [90]:
mining_df['Company_Name_NPRI'].value_counts()

Company_Name_NPRI
Vale Canada Limited                                   10
Cameco Corporation                                     4
Hudbay Minerals                                        4
ArcelorMittal Exploitation Minière Canada s.e.n.c.     3
Glencore Canada Corporation                            3
                                                      ..
Teck Metals Limited                                    1
Vale Canada Ltd                                        1
Vale Newfoundland and Labrador Limited                 1
Victoria Gold Corp.                                    1
Williams Operating Corporation                         1
Name: count, Length: 72, dtype: int64

In [91]:
manufacturing_df['Company_Name_NPRI'].value_counts()

Company_Name_NPRI
Rio Tinto Alcan Inc                            8
ArcelorMittal Produits Longs Canada s.e.n.c    4
EVRAZ Inc NA Canada                            4
Glencore Canada Corporation                    3
5N Plus Inc.                                   3
                                              ..
Umicore Precious Metals Canada Inc.            1
Valbruna ASW Inc.                              1
Vale Canada Ltd                                1
Vale Newfoundland and Labrador Limited         1
Wabtec Foundry Ltd.                            1
Name: count, Length: 133, dtype: int64

## 2.3 Add SUT energy accounts

In [52]:
sut_df = pd.read_csv(r'data/Emissions_accounts/Raw_data/Mining industries, energy consumption by NAICS - 16100029-eng/16100029.csv') 

In [53]:
sut_df

,REF_DATE,GEO,DGUID,North American Industry Classification System (NAICS),Energy types,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2019,Canada,2021A000011124,Metal ore mining [2122],"Total, energy expenses (thousands of dollars)",Thousands of dollars,284,units,0,v1354185599,1.1.1,1929480.0,NaN,NaN,NaN,0
1,2019,Canada,2021A000011124,Metal ore mining [2122],"Coal, quantity purchased (tonnes)",Tonnes,287,units,0,v1354185610,1.1.2,207878.0,NaN,NaN,NaN,0
2,2019,Canada,2021A000011124,Metal ore mining [2122],"Coal, expenses (thousands of dollars)",Thousands of dollars,284,units,0,v1354185621,1.1.3,35852.0,NaN,NaN,NaN,0
3,2019,Canada,2021A000011124,Metal ore mining [2122],"Petroleum coke, quantity purchased (tonnes)",Tonnes,287,units,0,v1354185627,1.1.4,0.0,NaN,NaN,NaN,0
4,2019,Canada,2021A000011124,Metal ore mining [2122],"Petroleum coke, expenses (thousands of dollars)",Thousands of dollars,284,units,0,v1354185628,1.1.5,0.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,2022,Canada,2021A000011124,"Shale, clay and refractory mineral mining and ...","Wood, quantity purchased (metric tonnes)",Metric tonnes,214,units,0,v1354186268,1.27.30,0.0,NaN,NaN,NaN,0
3668,2022,Canada,2021A000011124,"Shale, clay and refractory mineral mining and ...","Wood, expenses (thousands of dollars)",Thousands of dollars,284,units,0,v1354186269,1.27.31,0.0,NaN,NaN,NaN,0
3669,2022,Canada,2021A000011124,"Shale, clay and refractory mineral mining and ...","Water, quantity purchased (litres)",Litres,203,units,0,v1354186270,1.27.32,NaN,x,NaN,NaN,0
3670,2022,Canada,2021A000011124,"Shale, clay and refractory mineral mining and ...","Water, expenses (thousands of dollars)",Thousands of dollars,284,units,0,v1354186271,1.27.33,NaN,x,NaN,NaN,0


In [54]:
def clean_sut(df, year=None, naics=None, units_to_exclude=None):
    # Drop specified columns
    columns_to_remove = ['DGUID', 'UOM_ID', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS', 'SCALAR_FACTOR']
    df_cleaned = df.drop(columns=columns_to_remove, errors='ignore')
    
    # Filter by year if specified
    if year is not None:
        df_cleaned = df_cleaned[df_cleaned['REF_DATE'] == year]
    
    # Filter by NAICS if specified
    if naics is not None:
        df_cleaned = df_cleaned[df_cleaned['North American Industry Classification System (NAICS)'] == naics]
    
    # Exclude specified units from the UOM column if provided
    if units_to_exclude is not None:
        df_cleaned = df_cleaned[~df_cleaned['UOM'].isin(units_to_exclude)]
        
    # Remove rows where VALUE is 0 or NaN
    df_cleaned = df_cleaned[df_cleaned['VALUE'].notna() & (df_cleaned['VALUE'] != 0)]
    
    df_cleaned.reset_index(drop=True, inplace=True)
        
    return df_cleaned

In [55]:
sut_df_cleaned = clean_sut(sut_df, year=2022, units_to_exclude=["Thousands of dollars"])
sut_df_cleaned

,REF_DATE,GEO,North American Industry Classification System (NAICS),Energy types,UOM,VALUE
0,2022,Canada,Metal ore mining [2122],"Coal, quantity purchased (tonnes)",Tonnes,179231.0
1,2022,Canada,Metal ore mining [2122],"Natural gas, quantity purchased (cubic metres)",Cubic metres,152461339.0
2,2022,Canada,Metal ore mining [2122],"Gasoline - aviation, quantity purchased (litres)",Litres,11834517.0
3,2022,Canada,Metal ore mining [2122],"Gasoline - motor, quantity purchased (litres)",Litres,31567077.0
4,2022,Canada,Metal ore mining [2122],"Diesel fuel, quantity purchased (litres)",Litres,941997194.0
...,...,...,...,...,...,...
117,2022,Canada,"Sand and gravel, mining and quarrying [212323]","Light Fuel Oil, quantity purchased (litres)",Litres,160123.0
118,2022,Canada,"Sand and gravel, mining and quarrying [212323]","Heavy Fuel Oil, quantity purchased (litres)",Litres,424742.0
119,2022,Canada,"Sand and gravel, mining and quarrying [212323]","Propane, quantity purchased (litres)",Litres,1152653.0
120,2022,Canada,"Shale, clay and refractory mineral mining and ...","Diesel fuel, quantity purchased (litres)",Litres,577582.0


In [56]:
sut_df_cleaned['North American Industry Classification System (NAICS)'].value_counts()

North American Industry Classification System (NAICS)
Metal ore mining [2122]                                                                                        10
Sand, gravel, clay, and ceramic and refractory minerals mining and quarrying [21232]                            9
Other non-metallic mineral mining and quarrying [21239]                                                         8
Non-metallic mineral mining (excluding aggregates, clay and refractory mineral mining and quarrying) [2123]     8
Aggregates, clay and refractory mineral mining and quarrying (excluding non-metallic mineral mining) [2123]     8
Stone mining and quarrying [21231]                                                                              8
Copper, nickel, lead and zinc ore mining [21223]                                                                7
Copper-zinc ore mining [212233]                                                                                 7
Limestone mining and quarrying [21

In [57]:
def split_by_naics(df_cleaned):
    # Group the DataFrame by the 'NAICS' column
    naics_groups = df_cleaned.groupby('North American Industry Classification System (NAICS)')
    
    # Create a dictionary where keys are NAICS values and values are the corresponding DataFrames
    naics_dfs = {naics: group.reset_index(drop=True) for naics, group in naics_groups}
    
    return naics_dfs

In [58]:
# Apply the function to the cleaned DataFrame
naics_dfs = split_by_naics(sut_df_cleaned)

# Access a specific NAICS DataFrame
uranium_mining_df = naics_dfs['Uranium ore mining [212291]']

In [59]:
uranium_mining_df

,REF_DATE,GEO,North American Industry Classification System (NAICS),Energy types,UOM,VALUE
0,2022,Canada,Uranium ore mining [212291],"Diesel fuel, quantity purchased (litres)",Litres,2958363.0
1,2022,Canada,Uranium ore mining [212291],"Propane, quantity purchased (litres)",Litres,29071811.0
2,2022,Canada,Uranium ore mining [212291],"Electricity, quantity purchased (kilowatt-hour)",Kilowatt-hours,305354512.0


# 3 - Normalize emissions per facility

In [26]:
df_production = pd.read_excel(r'../data/canada_site_specific_data.xlsx')

PermissionError: [Errno 13] Permission denied: '../data/canada_site_specific_data.xlsx'

## Example 

In [38]:
npri_voisey = dfs_by_company_facility["Vale_Newfoundland_and_Labrador_Limited_Voisey's_Bay_Mine_Site"]
npri_voisey

,level_0,index,Year,NPRI ID,Company Name,Facility Name,City,CSD,CA or CMA,Economic Region,...,Metal Recovery,Inorganic Compound Recovery,Acid or Base Recovery,Catalyst Recovery,Pollution Abatement Residue Recovery,Used Oil Recovery,Other.2,Total.5,"Total Releases, Disposals and Transfers for Recycling",EI
4,32,1070,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,384.221000,Ammonia
11,100,1714,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1421.578000,Arsenic
25,191,3009,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1604.145000,Cadmium
39,258,3672,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.124000,"Carbon monoxide, fossil"
54,329,7468,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563.493000,Chromium
61,391,7957,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241743.578000,Cobalt
74,464,8340,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1237.842000,Copper
87,541,9147,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,No match found
89,553,9702,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,56.975,56.975,56.975000,No match found
91,571,10468,2023,6093,Vale Newfoundland and Labrador Limited,Voisey's Bay Mine Site,Happy Valley-Goose Bay,"Division No. 10, Subd. E",,West Coast--Northern Peninsula--Labrador / Côt...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.808000,"Benzene, hexachloro-"


In [33]:
def allocate_emissions(
    df, production_data, metals, economic_values=None, method='mass',
    unit_column='Units', 
):
    """
    Updated function to exclude specific columns like 'EI' from conversion,
    while performing unit conversion, normalization, and allocation.
    """
    # Step 1: Trim the DataFrame to include only relevant columns
    start_col = 'Substance Name (English)'
    df = df.loc[:, start_col:].copy()

    # Step 2: Identify emission columns (exclude non-numeric columns like 'EI')
    start_col = 'Stack Emissions'
    end_col = 'Total Releases, Disposals and Transfers for Recycling'
    emission_columns = df.loc[:, start_col:end_col].select_dtypes(include='number').columns.tolist()

    # Step 3: Convert units to kg for emissions columns only
    conversion_factors = {'tonnes': 1000, 'kg': 1, 'grams': 0.001, 'g TEQ': 0.001}
    if unit_column in df.columns:
        for unit, factor in conversion_factors.items():
            mask = df[unit_column] == unit
            df.loc[mask, emission_columns] = df.loc[mask, emission_columns] * factor
        df[unit_column] = 'kg'

    # Step 4: Normalize emissions per kg of total production
    total_production = sum(production_data.values())
    for col in emission_columns:
        df[f'{col}_per_kg_Total'] = df[col] / total_production

    # Step 5: Allocate emissions (mass or economic)
    allocation_factors = {}
    for metal, production in production_data.items():
        if method == 'mass':
            allocation_factors[metal] = production / total_production
        elif method == 'economic':
            total_economic_value = sum(production_data[m] * economic_values[m] for m in metals)
            allocation_factors[metal] = (production * economic_values[metal]) / total_economic_value
        else:
            raise ValueError("Invalid method. Use 'mass' or 'economic'.")

    for col in emission_columns:
        for metal in metals:
            df[f'{col}_per_kg_{metal}'] = df[f'{col}_per_kg_Total'] * allocation_factors[metal]

    return df

In [34]:
production_data = {'Nickel': 50000000, 'Copper': 32000000}  # kg of nickel and copper metal in concentrate
economic_values = {'Nickel': 13800, 'Copper': 6160}  # unit value in 2019 $/t given by USGS
metals = ['Nickel', 'Copper']

In [35]:
mass_df = allocate_emissions(
    npri_voisey, production_data, metals, economic_values=economic_values, method='mass', 
    unit_column='Units'
)

economic_df = allocate_emissions(
    npri_voisey, production_data, metals, economic_values=economic_values, method='economic', 
    unit_column='Units'
)

C:\Users\mp_ma\AppData\Local\Temp\ipykernel_6360\1067121282.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_per_kg_{metal}'] = df[f'{col}_per_kg_Total'] * allocation_factors[metal]
C:\Users\mp_ma\AppData\Local\Temp\ipykernel_6360\1067121282.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_per_kg_{metal}'] = df[f'{col}_per_kg_Total'] * allocation_factors[metal]
C:\Users\mp_ma\AppData\Local\Temp\ipykernel_6360\1067121282.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [36]:
def organize_emissions_data(df, metals):
    """
    Organize the emissions data into a clean format with desired columns.
    """
    # Define category mapping
    category_mapping = {
        'Air Emissions': ['Stack Emissions', 'Storage / Handling', 'Fugitive Emissions', 'Spills', 'Other', 'Total'],
        'Water Releases': ['Direct Discharge', 'Spills', 'Leaks', 'Total'],
        'Land Releases': ['Spills', 'Leaks', 'Other', 'Total'],
        'Total Releases': ['Total Releases (Excluding Road Dust)', 'Road Dust Emissions', 'Total Releases Including Road Dust'],
        'On-Site Disposal': ['Land Fill', 'Land Treatment', 'Underground Injection', 'Tailings', 'Waste Rock', 'Total On-Site'],
        'Off-Site Disposal': ['Land Fill', 'Land Treatment', 'Underground Injection', 'Storage', 'Tailings', 'Waste Rock', 'Total Off-Site'],
        'Transfers for Recycling': ['Energy Recovery', 'Metal Recovery', 'Other', 'Total']
    }

    # Prepare rows for clean output
    rows = []
    for category, columns in category_mapping.items():
        for col in columns:
            total_col = f"{col}_per_kg_Total"
            metal_cols = [f"{col}_per_kg_{metal}" for metal in metals]

            if total_col in df.columns:
                for _, row in df.iterrows():
                    row_data = {
                        'Type': category,
                        'Category': col,
                        'Substance Name (English)': row['Substance Name (English)'],
                        'Ecoinvent Name': row['EI'] if 'EI' in df.columns else 'Unknown',
                        'Unit': row['Units'],
                        'Total Emissions (per kg)': row[total_col],
                    }
                    for metal, metal_col in zip(metals, metal_cols):
                        row_data[f'{metal} Emissions (per kg)'] = row[metal_col] if metal_col in df.columns else 0
                    rows.append(row_data)

    # Convert to DataFrame
    result_df = pd.DataFrame(rows)

    return result_df

In [37]:
mass_df = organize_emissions_data(mass_df, metals)
economic_df = organize_emissions_data(economic_df, metals)

In [38]:
with pd.ExcelWriter('allocation.xlsx', engine='xlsxwriter') as writer:
    mass_df.to_excel(writer, sheet_name='mass', index=False)
    economic_df.to_excel(writer, sheet_name='eco', index=False) 

# 4 - Create national consumption markets based on production data 